In [1]:
import torch
from skimage import io
from skimage import io
import matplotlib.pyplot as plt
import cv2

%matplotlib inline

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # or yolov5m, yolov5l, yolov5x, custom

In [7]:
Path      = "archive/"
#Path      = "/content/drive/MyDrive/152_478_AI_2564_2/video_data/"    
filename  = "WADADA .mp4"


print(Path+filename)

archive/WADADA .mp4


In [8]:
# capture the video
cap = cv2.VideoCapture(Path+filename)

# check if capture was successful
if not cap.isOpened(): 
    print("Could not open!")
else:
    print("Video read successful!")
    
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width        = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height       = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps          = cap.get(cv2.CAP_PROP_FPS)
    
    print('Total frames: ' + str(total_frames))
    print('width: ' + str(width))
    print('height: ' + str(height))
    print('fps: ' + str(fps))

Could not open!


In [ ]:
savepath = ""
savepath = "/content/drive/MyDrive/152_478_AI_2564_2/video_data/"
savefile = 'Time_stamp_output.mp4'

fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out    = cv2.VideoWriter(savepath+savefile, fourcc, fps, (width,height))
    


cap = cv2.VideoCapture(filename)


for i in range(1000):

    cap.set(cv2.CAP_PROP_POS_FRAMES, i)    

   # Get the next frame, but don't decode if we're not using it.
    success    = cap.grab()

    ret, image = cap.retrieve()

    if  ret :
      duration = i/fps
      text = f"FrameNo {i} Time = {duration:.2f}"

      start_point = (20,height-30)
      image = cv2.putText(image, text, start_point, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA, False)
      out.write(image)


    
    if i%100 ==0 :
        plt.figure(figsize=(10, 10))
        plt.imshow(image)
        plt.show()


out.release()      
cap.release()
